<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/%E9%80%94%E4%B8%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import 
import torch 
import torch.nn as nn 
import torch .optim as optim 
import torch.utils.data as data 
import torchvision 
from torchvision import models, transforms 

import time
import glob 
import os.path as osp 
import random 
import numpy as np 
import json 
from tqdm import tqdm
import matplotlib.pyplot as plt 
%matplotlib inline 
from PIL import Image  

In [0]:
class Generator(nn.Module):
  def __init__(self,z_dim=20,image_size=64):
    super(Generator,self).__init__()

    self.layer1 = nn.Sequential(
        nn.ConvTranspose2d(z_dim,image_size * 8, kernel_size=4,stride=1),
        nn.BatchNorm2d(image_size*8),
        nn.ReLU(inplace=True)
    )
    self.layer2 = nn.Sequential(
        nn.ConvTranspose2d(image_size * 8,image_size*4, kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(image_size*4),
        nn.ReLU(inplace=True)
    )
    self.layer3 = nn.Sequential(
        nn.ConvTranspose2d(image_size * 4,image_size*2, kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(image_size*2),
        nn.ReLU(inplace=True)
    )
    self.layer4 = nn.Sequential(
        nn.ConvTranspose2d(image_size * 2,image_size, kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(image_size),
        nn.ReLU(inplace=True)
    )
    self.last=nn.Sequential(
        nn.ConvTranspose2d(image_size, 1, kernel_size=4,stride=2,padding = 1),
    nn.Tanh()
    )

  def forward(self,z):
    out = self.layer1(z)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.last(out)
    return out 

In [0]:
def make_datapath_list_sub():
  rootpath = '/content/drive/My Drive/Ironman/output/'
  target_path = osp.join(rootpath+'*.jpg')
  print(target_path)
  
  path_list = []
  
  #globを利用してサブディレクトリまでパスを取得する
  for path in glob.glob(target_path):
    path_list.append(path)
  
  return path_list

In [83]:
train_img_list = make_datapath_list_sub()

/content/drive/My Drive/Ironman/output/*.jpg


In [84]:
print(len(train_img_list))
train_img_list

107


['/content/drive/My Drive/Ironman/output/21_resize.jpg',
 '/content/drive/My Drive/Ironman/output/20_resize.jpg',
 '/content/drive/My Drive/Ironman/output/15_resize.jpg',
 '/content/drive/My Drive/Ironman/output/14_resize.jpg',
 '/content/drive/My Drive/Ironman/output/16_resize.jpg',
 '/content/drive/My Drive/Ironman/output/17_resize.jpg',
 '/content/drive/My Drive/Ironman/output/19_resize.jpg',
 '/content/drive/My Drive/Ironman/output/18_resize.jpg',
 '/content/drive/My Drive/Ironman/output/1_resize.jpg',
 '/content/drive/My Drive/Ironman/output/13_resize.jpg',
 '/content/drive/My Drive/Ironman/output/12_resize.jpg',
 '/content/drive/My Drive/Ironman/output/11_resize.jpg',
 '/content/drive/My Drive/Ironman/output/103_resize.jpg',
 '/content/drive/My Drive/Ironman/output/100_resize.jpg',
 '/content/drive/My Drive/Ironman/output/102_resize.jpg',
 '/content/drive/My Drive/Ironman/output/104_resize.jpg',
 '/content/drive/My Drive/Ironman/output/105_resize.jpg',
 '/content/drive/My Drive/I

In [0]:
def make_datapath_list():
  train_img_list = list()   #画像ファイルパスを格納

  for img_idx in range(55):
    img_path = '/content/drive/My Drive/Ironman/output/' + str(img_idx) + '_resize.jpg'
    train_img_list.append(img_path)


  return train_img_list

In [0]:
class ImageTransform():
  #画像の前処理クラス

  def __init__(self,mean,std):
    self.data_transform = transforms.Compose([
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean,std)
    ])
  def __call__(self,img):
    return self.data_transform(img)

In [87]:
class GAN_Img_Dataset(data.Dataset):
  #画像のDatasetクラス.PytorchのDatasetクラスを継承
  def __init__(self,file_list,transform):
    self.file_list = file_list
    self.transform = transform
  
  def __len__(self):
    #画像の枚数を返す
    return len(self.file_list)
    
  def __getitem__(self,index):
    #前処理した画像のtensor形式のデータを取得
    img_path = self.file_list[index]
    img = Image.open(img_path)

    #画像の前処理
    img_transformed = self.transform(img)

    return img_transformed 

#DataLoader の動作と確認
#ファイルリストを作成
train_img_list = make_datapath_list()

#Datasetを作成
mean = (0.5,)
std = (0.5,)
train_dataset = GAN_Img_Dataset(
    file_list = train_img_list,transform=ImageTransform(mean,std)
)

#DataLoaderを作成
batch_size = 64 
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,batch_size=batch_size,shuffle=True
)

#動作の確認
batch_iterator = iter(train_dataloader)
images = next(batch_iterator)
print(images.size())

RuntimeError: ignored